In [1]:
import pandas as pd
import os
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

df = pd.read_csv(r"csv/akakce.csv")

# Fiyat sütununu temizle
df["Price"] = df["Price"].apply(
    lambda x: int(str(x).replace(" TL", "").split(",")[0].replace(".", ""))
)
df["Price"] = df["Price"].astype(int)

# Türkiye tarih formatı: 'gün.ay.yıl'
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y", errors="coerce")

# Aynı ürün ve aynı tarih varsa sadece bir tanesini tut
df = df.sort_values(["Product Name", "Date"])
df = df.drop_duplicates(subset=["Product Name", "Date"]).reset_index(drop=True)

# Her bir ürün için grafik çiz
product_names = df["Product Name"].unique()

output_dir = "plots"
os.makedirs(output_dir, exist_ok=True)  # Grafiklerin kaydedileceği klasör
print(len(product_names))
for product in product_names:
    product_df = df[df["Product Name"] == product].sort_values("Date")

    # Grafik çiz
    plt.figure(figsize=(12, 6))
    plt.plot(product_df["Date"], product_df["Price"], marker="o")
    plt.title(f"{product} - Fiyat Zaman Serisi")
    plt.xlabel("Tarih")
    plt.ylabel("Fiyat (TL)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()

    # Grafik dosyasını kaydet (boşluklar yerine alt çizgi kullan)
    filename = f"{output_dir}/{product.replace(' ', '_')}.png"
    #plt.savefig(filename)
    #plt.show()
    #plt.close()

    print(f"Grafik kaydedildi: {filename}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings
import os

warnings.filterwarnings("ignore")

# Veriyi yükle
df = pd.read_csv(r"csv/akakce.csv")

# Fiyat sütununu temizle
df["Price"] = df["Price"].apply(
    lambda x: int(str(x).replace(" TL", "").split(",")[0].replace(".", ""))
)
df["Price"] = df["Price"].astype(int)

# Tarih sütununu dönüştür
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y", errors="coerce")

# Aynı ürün ve aynı tarih varsa sadece bir tanesini tut
df = df.sort_values(["Product Name", "Date"])
df = df.drop_duplicates(subset=["Product Name", "Date"]).reset_index(drop=True)

# Ürün isimlerini al
product_names = df["Product Name"].unique()

# Tahminlerin kaydedileceği klasör
output_dir = "predictions"
os.makedirs(output_dir, exist_ok=True)

# Her ürün için döngü
for product in product_names:
    product_df = df[df["Product Name"] == product].sort_values("Date")

    if len(product_df) < 10:
        print(f"Yetersiz veri: {product} (atlandı)")
        continue

    try:
        model = ARIMA(product_df["Price"], order=(3, 1, 1))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=5)

        # Grafik
        plt.figure(figsize=(10, 5))
        plt.plot(product_df["Date"], product_df["Price"], label="Gerçek Fiyat")
        future_dates = pd.date_range(
            start=product_df["Date"].iloc[-1], periods=6, freq="D"
        )[1:]
        plt.plot(future_dates, forecast, label="Tahmin", color="red", marker="o")
        plt.title(f"{product} - 5 Günlük Fiyat Tahmini")
        plt.xlabel("Tarih")
        plt.ylabel("Fiyat (TL)")
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        # Kaydet
        filename = f"{output_dir}/{product.replace(' ', '_')}_forecast.png"
        plt.savefig(filename)
        plt.close()
        print(f"Tahmin grafiği kaydedildi: {filename}")

    except Exception as e:
        print(f"Hata oluştu ({product}): {e}")

In [ ]:
import matplotlib.pyplot as plt



In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['Price'])  # Zaman serisi burada 'fiyat'
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
df['price_diff'] = df['Price'].diff().dropna()
df.dropna(subset=['price_diff'], inplace=True)
print(len(df))
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['price_diff'])
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")



In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

model = ARIMA(df['Price'], order=(1, 1, 1)) 
model_fit = model.fit()
forecast = model_fit.forecast(steps=10)
print(forecast)
plt.plot(df['Price'], label='Gerçek')
plt.plot(range(len(df), len(df)+10), forecast, label='Tahmin', color='red')
plt.title('ARIMA Tahmini')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

data = pd.DataFrame({
    'RAM': [8, 6, 4, 8, 16],
    'Depolama': [128, 128, 64, 256,512],
    'Ekran': [6.5, 6.3, 5.8, 6.7,6.7],
    'Kamera': [64, 48, 12, 108,200],
    'Fiyat': [17000, 19000, 20000, 25000,40000]
}

)

X = data[['RAM', 'Depolama', 'Ekran', 'Kamera']]
y = data['Fiyat']

model = KNeighborsRegressor(n_neighbors=2)
model.fit(X, y)

yeni_telefon = pd.DataFrame({
    'RAM': [6],
    'Depolama': [64],
    'Ekran': [5.2],
    'Kamera': [5]
})

tahmin = model.predict(yeni_telefon)
print(f"Estimation price: {tahmin[0]:.2f} TL")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor

data = pd.DataFrame({
    'RAM':       [2, 4, 6, 8, 12, 4, 6, 8, 12, 16, 3, 6, 8, 8, 4, 6, 10, 12, 6, 8],
    'Storage':  [32, 64, 128, 128, 256, 64, 128, 128, 256, 512, 64, 128, 128, 256, 64, 128, 256, 512, 128, 256],
    'Display':     [5.0, 5.5, 6.1, 6.3, 6.7, 5.8, 6.0, 6.4, 6.5, 6.8, 5.2, 6.2, 6.3, 6.6, 5.5, 6.1, 6.6, 6.9, 6.3, 6.7],
    'Camera':    [8, 12, 16, 48, 64, 13, 20, 32, 64, 200, 5, 24, 48, 108, 12, 16, 64, 200, 50, 64],
    'Price':     [3000, 4500, 7000, 10000, 14000, 5000, 7500, 9000, 13000, 40000, 2500, 6800, 9800, 17000, 4900, 7200, 12000, 39000, 8500, 16000]
})

X = data[['RAM', 'Storage', 'Display', 'Camera']]
y = data['Price']

model = KNeighborsRegressor(n_neighbors=4)
model.fit(X, y)

yeni_telefon = pd.DataFrame({
    'RAM': [1],
    'Storage': [190],
    'Display': [5.2],
    'Camera': [2]
})

tahmin = model.predict(yeni_telefon)
print(f"Estimation price: {tahmin[0]:.2f} TL")
corr = data.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation matrix for Telephone Features")
plt.show()


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(r"csv/phones.csv")

X = df[["ram", "storage", "display_size", "weight"]]
y = df["price_usd"]

print(df["phone_model"].head(10))

df["phone_model"].to_csv(r"csv/phoneNames.csv", index=False)

model = KNeighborsRegressor(n_neighbors=4)
model.fit(X, y)

yeni_telefon = pd.DataFrame(
    [{"ram": 700, "storage": 1024, "display_size": 6.2, "weight": 199.0}]
)

tahmin = model.predict(yeni_telefon)
print(f"Estimation price: {tahmin[0]:.2f} USD")


numeric_df = df.select_dtypes(include=[float, int])

correlation_matrix = numeric_df.corr()
plt.figure(figsize=(10, 8))  
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

plt.title("Correlation matrix")
plt.show()

In [ ]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv(r"csv/trendyolyeni.csv")

# İlk 5 ürünü görüntüle
print(df["Description"].head(5))

# Ürün açıklamalarını ';' ile böl
product = df["Description"].str.split(";")


# Her ürün açıklamasını anahtar-değer çiftine dönüştürmek
def process_product_description(description):
    product_dict = {}
    for item in description:
        # ":" ile bölerek anahtar ve değeri ayır
        if ":" in item:
            key, value = item.split(":", 1)
            product_dict[key.strip()] = value.strip()  # boşlukları temizle
    return product_dict


# Her ürün için sözlük oluştur
product_list = product.map(process_product_description)

# Sonuçları yazdır
print(product_list.head(5))

product_df = pd.DataFrame(product_list.tolist())
product_df.to_csv("csv/parsed_trendyol.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

df = pd.read_csv(r"csv/phones.csv")

df = df.drop_duplicates(subset=["phone_model"]).reset_index(drop=True)

df["launch_date"] = pd.to_datetime(
    df["launch_date"], format="%m/%d/%Y", errors="coerce"
)

df = df[df["launch_date"] > "2023-01-20"].reset_index(drop=True)

print(df["phone_model"])

df["phone_model"].to_csv(r"csv/phones_after_2023.csv", index=False, encoding="utf-8-sig")